In [3]:
from glob import glob

for g in glob("data/*.pdf"):
    print(g)

data/2040_서울도시기본계획.pdf
data/OneNYC-2050-Summary.pdf


In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def read_pdf_and_split_text(pdf_path, chunk_size=1000, chunk_overlap=100):
    print(f"PDF: {pdf_path} -----------------------------------------")

    pdf_loader = PyPDFLoader(pdf_path)
    data_from_pdf = pdf_loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )

    splits = text_splitter.split_documents(data_from_pdf)

    print(f"PDF: {pdf_path} -----------------------------------------")
    print(f"Number of splits: {len(splits)}")

    return splits


In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import os

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

persist_directory = "./chroma_store"

if os.path.exists(persist_directory):
    print(f"Chroma directory {persist_directory} already exists")

    vectorestore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
    )
else:
    print(f"Chroma directory {persist_directory} does not exist")

    vectorstore = None
    for g in glob("data/*.pdf"):
        chunks = read_pdf_and_split_text(g)

        for i in range(0, len(chunks), 100):
            if vectorstore is None:
                vectorstore = Chroma.from_documents(
                    documents=chunks[i:i+100],
                    embedding=embeddings,
                    persist_directory=persist_directory,
                )
            else:
                vectorstore.add_documents(
                    documents=chunks[i : i + 100],
                )

Chroma directory ./chroma_store does not exist
PDF: data/2040_서울도시기본계획.pdf -----------------------------------------
PDF: data/2040_서울도시기본계획.pdf -----------------------------------------
Number of splits: 293
PDF: data/OneNYC-2050-Summary.pdf -----------------------------------------
PDF: data/OneNYC-2050-Summary.pdf -----------------------------------------
Number of splits: 176


In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

chuncks = retriever.invoke("서울 온실가스 저감 계획")

for chunk in chunks:
    print(chunk.metadata)
    print(chunk.page_content)
    print("--------------------------------")


{'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Windows)', 'creationdate': '2019-04-30T10:57:03-04:00', 'moddate': '2020-01-03T16:36:59-05:00', 'trapped': '/False', 'source': 'data/OneNYC-2050-Summary.pdf', 'total_pages': 60, 'page': 0, 'page_label': '1'}
OneNYC 
2050
BUILDING A STRONG 
AND FAIR CITY 
VOLUME 1 OF 9  
A
PRIL 2019
THE CITY OF NEW YORK
MAYOR BILL DE BLASIO
DEAN FULEIHAN  
FIRST DEPUTY MAYOR 
D
OMINIC WILLIAMS  
CHIEF POLICY ADVISOR
D
ANIEL A. ZARRILLI  
One NYC DIRECTOR
--------------------------------
{'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Windows)', 'creationdate': '2019-04-30T10:57:03-04:00', 'moddate': '2020-01-03T16:36:59-05:00', 'trapped': '/False', 'source': 'data/OneNYC-2050-Summary.pdf', 'total_pages': 60, 'page': 1, 'page_label': '2'}
2   |   O neNYC 2050 
NYC.GOV/O neNYC
ONENYC 2050 IS A STRATEGY TO SECURE OUR CITY’S 
FUTURE AGAINST THE CHALLENGES OF TODAY AND 
TOMORROW. WITH BOLD ACTIONS TO CONFRONT OUR 

In [8]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(api_key=api_key)
model.invoke("안녕하세요!")

AIMessage(content='안녕하세요! 도와드릴 수 있는게 있으면 말씀해주세요. :)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 13, 'total_tokens': 39, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C27swQY3botiTM0lQBPwSz9XK7BAQ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8e3840fa-0779-4bf7-adb6-5316b8aed40c-0', usage_metadata={'input_tokens': 13, 'output_tokens': 26, 'total_tokens': 39, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal
from pydantic import BaseModel, Field

class RouteQuery(BaseModel):
    datasource: Literal["vectorstore", "casual_talk"] = Field(
        ...,
        description=""
    )

In [10]:
structured_llm_router = model.with_structured_output(RouteQuery)

router_system = """
당신은 사용자의 질문을 vectorestore 또는 casual_talk 으로 라우팅하는 전문가입니다.
- vectorestore에는 서울, 뉴욕의 발전계획과 관련된 문서가 포함되어 있습니다. 이 주제에 대한 질문에는 vectorestore를 사용하십시오.
- 사용자의 질문이 일상 대화에 관련된 경우 casual_talk을 사용하십시오.
"""

route_prompt = ChatPromptTemplate.from_messages([
    ("system", router_system),
    ("human", "{question}")
])

question_router = route_prompt | structured_llm_router

/Users/tkhwang/Documents/git/_tkhwang/tkhwang-book-lecture-coding/ai/book-do-it-ai-agent/.venv/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1857: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [11]:
print(
    question_router.invoke({
        "question": "잘 지냈어 ?"
    })
)



datasource='casual_talk'


In [12]:
print(question_router.invoke({"question": "서울 온실가스 저감 계획은 무엇인가요 ?"}))

datasource='vectorstore'


In [13]:
from langchain_core.prompts import PromptTemplate

class GradeDocuments(BaseModel):
    binary_score: Literal["yes", "no"] = Field(
        description="문서가 질문과 관련이 있는지 여부를 'yes' 또는 'no'로 표시하세요."
    )

structured_llm_grader = model.with_structured_output(GradeDocuments)

grader_prompt = PromptTemplate.from_template("""
당신은 검색된 문서가 사용자 질문과 관련이 있는지 평가하는 평가자입니다.\n
문서에 사용자 질문과 관련된 키워드 또는 의미가 포함되어 있으면, 해당 문서를 관련성이 있다고 평가하십시오.\n
엄격한 테스트가 필요하지 않습니다. 목표는 잘못된 검색 결과를 걸러내는 것입니다.\n
문서가 질문과 관련이 있는지 여부를 나타내기 위해 'yes' 또는 'no'로 이진 점수를 부여하십시오.\ng

Retrieved documents: \n {document} \n\n
User question: {question}
""")

retrieval_grader = grader_prompt | structured_llm_grader
question = "서울시 자율주행 관련 계획"
documents = retriever.invoke(question)

for doc in documents:
    print(doc)



/Users/tkhwang/Documents/git/_tkhwang/tkhwang-book-lecture-coding/ai/book-do-it-ai-agent/.venv/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1857: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


page_content='확충한다.
- 자율주행 자동차 기반 구축, 서울형 도심항공 교통인프라 확보, 모빌리티 허브 및 3차원 물류체계 
구축 등 도시인프라 확충을 위한 기반 마련 필요
 신 교통수단과 기존 대중교통의 원활한 환승체계 구축을 위해 주요 거점에 광역형, 지역형, 근린형 
등 중심지 위계에 따른 유형별 환승체계를 계획한다.' metadata={'creationdate': '2023-02-14T16:54:21+09:00', 'total_pages': 206, 'source': 'data/2040_서울도시기본계획.pdf', 'page': 115, 'moddate': '2023-02-14T18:26:31+09:00', 'author': '', 'title': '', 'page_label': '116', 'producer': 'Acrobat Distiller 9.0.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2'}
page_content='54 제2장 미래상과 목표
5. 기술발전에 선제적 대응, ‘미래교통 인프라 구축’
1) 배경
미래교통수단 도입에 따른 도시 활동 변화 및 공간구조 재편 대비
 빠르게 진화하는 미래 신 교통수단은 도시공간에 새로운 변화를 일으킬 것이며, 도시 내 이동패턴의 
변화를 가져올 것으로 예측된다. 이러한 점을 고려하여 도시 내 다양한 통행행태가 공존하는 포용적인 
교통체계 구축을 준비해야 한다.
미래교통수단 정착을 위한 도시 인프라 확충지원 필요
 기존에 없던 새로운 교통은 통상 인프라 구축에 많은 시간이 소요되기 때문에 교통수단이 도입된 이
후에 준비하기보다는 개발단계에서부터 이를 지원하기 위한 교통인프라 구축 구상이 요구되며, 새
로운 도시 인프라 확충에 대한 도시계획적 지원이 필요하다.
2) 추진전략
자율주행 자동차 본격 운영 체계 구축
 자율주행은 현재 제일 가시화되고 있는 미래 교통기술로 시장의 주도권을 확보하기 위해 각국의 
노력이 집중되고 있